In [ ]:
import cv2 as cv

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.collections as mc
import matplotlib.patches as mp

from copy import deepcopy
from dataclasses import dataclass
from enum import Enum
from functools import total_ordering
from numpy.random import default_rng
from queue import Queue, PriorityQueue

from typing import *
from numpy.typing import NDArray

rng = default_rng(5843)

## Utilities

In [ ]:
def fix_color(src: NDArray) -> NDArray:
    return cv.cvtColor(src, cv.COLOR_BGR2RGB)

In [ ]:
def plot_image(
        img: NDArray, cmap: str | None = None, *,
        title: str | None = None, format: int = cv.COLOR_BGR2RGB
    ):

    ax: plt.Axes
    _, ax = plt.subplots(dpi=140)
    
    ax.set(xticks=[], yticks=[])

    if title is not None:
        ax.set_title(title)

    if cmap is None:
        ax.imshow(cv.cvtColor(img, format))
    else:
        ax.imshow(img, cmap, vmin=0, vmax=255)

In [ ]:
def plot_images(images: Sequence[tuple[str, NDArray] | None], columns: int = 3, *,
                cmap: str | None = None, title: str | None = None, format = cv.COLOR_BGR2RGB,
                cell_size: tuple[float, float]=(4., 3.), dpi=120) -> None:
    
    images = list(images)
    rows = (len(images) + columns) // columns
    fig, axs = plt.subplots(rows, columns, figsize=(columns * cell_size[0], rows * cell_size[1]),
                            layout='constrained', dpi=dpi)

    if title is not None:
        fig.suptitle(title)
    
    ax: plt.Axes
    for ax in axs.flat:
        ax.set_visible(False)
    
    for i, entry in enumerate(images):
        if entry is None: continue
        subtitle, img = entry

        ax: plt.Axes = axs.flat[i]
        ax.set(xticks=[], yticks=[], title=subtitle, visible=True)
        if img.shape.count == 2:
            ax.imshow(img, cmap=cmap, vmin=0, vmax=255)
        else:
            ax.imshow(cv.cvtColor(img, format))

## Detect Grid Shape

In [ ]:
image_name = 'uni-1_05x04_shuffled_948135.jpg'
image = cv.imread('../images/' + image_name)
plot_image(image, title=image_name)

In [ ]:
image_gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
plot_image(image_gray, 'gray', title=f'{image_name} in Grayscale')

In [ ]:
sobel_horizontal = cv.Sobel(image_gray, cv.CV_32F, 1, 0)
plot_image(np.abs(sobel_horizontal), 'magma', title=f'Horizontal Sobel of {image_name}')

In [ ]:
sobel_vertical = cv.Sobel(image_gray, cv.CV_32F, 0, 1)
plot_image(np.abs(sobel_vertical), 'magma', title=f'Vertical Sobel of {image_name}')

In [ ]:
def plot(y: NDArray, title: str | None = None):
    fig, ax = plt.subplots(figsize=(18, 5))
    ax: plt.Axes

    length = y.shape[0]
    mean = np.average(y)
    variance = np.average(np.square(mean - y))
    std = np.sqrt(variance)

    ax.set(xlim=[0, length], title=title)
    
    ax.plot(y, label='y')
    ax.axhline(mean+std*4, c='C4', label=r'$\mu + 4\sigma$')
    ax.axhline(mean+std*2, c='C3', label=r'$\mu + 2\sigma$')
    ax.axhline(mean+std, c='C2', label=r'$\mu + \sigma$')
    ax.axhline(mean, c='C1', label=r'$\mu$')

    ax.grid(ls=':')
    ax.legend()

average_horizontal = np.average(np.abs(sobel_horizontal), axis=0)
plot(average_horizontal, title=f'Average Horizontal Sobel of {image_name}')

In [ ]:
average_vertical = np.average(np.abs(sobel_vertical), axis=1)
plot(average_vertical, title=f'Average Vertical Sobel of {image_name}')

In [ ]:
plot(np.diff(average_horizontal), title=f'Derivative of Average Horizontal Sobel of {image_name}')

In [ ]:
plot(np.diff(average_vertical), title=f'Derivative of Average Vertical Sobel of {image_name}')

In [ ]:
def calc_std(y: NDArray) -> float:
    mean = np.average(y)
    variance = np.average(np.square(y - mean))
    return np.sqrt(variance)

threshold = 3

edges_horizontal = cv.threshold(
    np.diff(average_horizontal), calc_std(average_horizontal) * threshold,
    1, cv.THRESH_BINARY,
)[1]

edges_vertical = cv.threshold(
    np.diff(average_vertical), calc_std(average_vertical) * threshold,
    1, cv.THRESH_BINARY,
)[1]

In [ ]:
longest_gap = 50
edges_horizontal = cv.morphologyEx(edges_horizontal, cv.MORPH_CLOSE, np.ones(longest_gap))
edges_vertical = cv.morphologyEx(edges_vertical, cv.MORPH_CLOSE, np.ones(longest_gap))

In [ ]:
def length_encode(seq: NDArray) -> Iterator[tuple[bool, int]]:
    last, counter = next(seq.flat), 1

    for element in seq.flat:
        if last == element:
            counter += 1
        else:
            yield last, counter
            last, counter = element, 1
    
    yield last, counter


', '.join(map(lambda x: str(x[1]), length_encode(edges_vertical > .5)))

In [ ]:
def get_edge_length(seq: Iterator[tuple[bool, int]]) -> int:
    last_padding = 0
    last_length = -1
    min_length = -1

    for value, length in seq:
        if value and last_length != -1:
            padding = length // 2
            edge_length = last_length + last_padding + padding
            if edge_length < min_length and edge_length > longest_gap:
                min_length = edge_length

            last_padding = length - padding
            last_length = -1

        elif value:
            last_padding = length

        else:
            last_length = length

    if last_length != -1:
        edge_length = last_padding + last_length
        
        if edge_length < min_length and edge_length > longest_gap:
            min_length = edge_length
    
    return edge_length


cell_rows = get_edge_length(length_encode(edges_vertical > .5))
cell_columns = get_edge_length(length_encode(edges_horizontal > .5))

cell_rows, cell_columns

In [ ]:
rows = int(image.shape[0] / cell_rows + .5)
columns = int(image.shape[1] / cell_columns + .5)

rows, columns